In [2]:
#pip install Flask-PyMongo

In [29]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame (Extract)
* Pandas is used in the extraction process to read the CSV into a DataFrame.

In [30]:
csv_file = "../Resources/customer_data.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.head()

,id,first_name,last_name,email,gender,car
0,1,Benetta,Cancott,bcancott0@studiopress.com,Female,Scion
1,2,Lilyan,Cherry,lcherry1@deliciousdays.com,Female,Chrysler
2,3,Ezekiel,Benasik,ebenasik2@wikia.com,Male,Mercedes-Benz
3,4,Kennedy,Atlay,katlay3@so-net.ne.jp,Male,Buick
4,5,Sanford,Salmen,ssalmen4@reuters.com,Male,Lincoln


### Create new data with select columns (Transform)
* Pandas is again used in the transform process to clean the data to take in the columns that are needed for the `customer_name table`.

In [31]:
new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
new_customer_data_df.head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Store JSON data into a DataFrame (Extract)

In [32]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,address,id,latitude,longitude,us_state
0,043 Mockingbird Place,1,39.1682,-86.5186,Indiana
1,4 Prentice Point,2,41.0938,-85.0707,Indiana
2,46 Derek Junction,3,32.7673,-96.7776,Texas
3,11966 Old Shore Place,4,39.0350,-94.3567,Missouri
4,5 Evergreen Circle,5,40.7808,-73.9772,New York


### Clean DataFrame (Transform)

In [33]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Connect to local database
* The user will be postgres and the password will be the same one used to connect to pgAdmin 4 server.
* You may need to `pip install psycopg2` package to connect to pgAdmin 4.

In [34]:
#!pip install psycopg2

* The connection string consists of `user:password@host/db_name`. The user will be postgres and the password will be the same one used to connect to pgAdmin 4 server. 

* The host will be `localhost:5432`. This is then passed into the create_engine function, which starts with `postgresql://` followed by the connection string
* The postgresql:// specifies to SQLALchemy the type of the database connection. This can take other parameters such as mysql and sqlite.

In [35]:
rds_connection_string = "postgres:pgPassW@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables
* `engine.table_names()` will confirm a successful connection by returning the names of the tables in the pgAdmin 4 database.

In [38]:
engine.table_names()

['customer_location', 'customer_name', 'premise', 'county']

### Use pandas to load csv converted DataFrame into database

In [39]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "id" of relation "customer_name" does not exist
LINE 1: INSERT INTO customer_name (id, first_name, last_name) VALUES...
                                   ^

[SQL: INSERT INTO customer_name (id, first_name, last_name) VALUES (%(id)s, %(first_name)s, %(last_name)s)]
[parameters: ({'id': 1, 'first_name': 'Benetta', 'last_name': 'Cancott'}, {'id': 2, 'first_name': 'Lilyan', 'last_name': 'Cherry'}, {'id': 3, 'first_name': 'Ezekiel', 'last_name': 'Benasik'}, {'id': 4, 'first_name': 'Kennedy', 'last_name': 'Atlay'}, {'id': 5, 'first_name': 'Sanford', 'last_name': 'Salmen'}, {'id': 6, 'first_name': 'Ricki', 'last_name': 'Sheach'}, {'id': 7, 'first_name': 'Haroun', 'last_name': 'Cockitt'}, {'id': 8, 'first_name': 'Tuesday', 'last_name': 'Bortolozzi'}  ... displaying 10 of 1000 total bound parameter sets ...  {'id': 999, 'first_name': 'Benny', 'last_name': 'Wafer'}, {'id': 1000, 'first_name': 'Addia', 'last_name': 'Sedgebeer'})]
(Backgr

### Use pandas to load json converted DataFrame into database

In [40]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "id" of relation "customer_location" does not exist
LINE 1: INSERT INTO customer_location (id, address, us_state) VALUES...
                                       ^

[SQL: INSERT INTO customer_location (id, address, us_state) VALUES (%(id)s, %(address)s, %(us_state)s)]
[parameters: ({'id': 1, 'address': '043 Mockingbird Place', 'us_state': 'Indiana'}, {'id': 2, 'address': '4 Prentice Point', 'us_state': 'Indiana'}, {'id': 3, 'address': '46 Derek Junction', 'us_state': 'Texas'}, {'id': 4, 'address': '11966 Old Shore Place', 'us_state': 'Missouri'}, {'id': 5, 'address': '5 Evergreen Circle', 'us_state': 'New York'}, {'id': 6, 'address': '5304 Vidon Drive', 'us_state': 'Texas'}, {'id': 7, 'address': '65 Texas Avenue', 'us_state': 'Ohio'}, {'id': 8, 'address': '041 Eagle Crest Court', 'us_state': 'Texas'}  ... displaying 10 of 1000 total bound parameter sets ...  {'id': 999, 'address': '33 Bunting Point', 'us_state': 'Nevada'}, {'i

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [18]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [11]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
